## LinkedIn Profile Scrapper in Python

In [1]:
import requests, time, random
from bs4 import BeautifulSoup 
from selenium import webdriver
import pandas as pd
import operator
import pyodbc
import requests
import time
from datetime import datetime

In [27]:
#VARIABLES
#VARIABLES DE CONFIGURACION INICIAL

SESION_PC = 'ALEX'
TABLA_BUSCAR = 'parque_linkedin'
TABLA_BUSCAR2 = 'mae_titular_uba'

TABLA_INSERTAR = 'AV_BASE_LINKEDIN'
SERVER = 'SVRBI'
DB = 'BDANALYTICS'
HOY = datetime.now().strftime('%Y-%m-%d')
linkRef = []

In [3]:
def slideBar(num, browser):
    SCROLL_PAUSE_TIME = 10

    # Get scroll height
    #last_height = browser.execute_script("return document.body.scrollHeight")

    for i in range(num):
        # Scroll down to bottom
        if i == 0: #PRIMERO IRA A LA TERCERA PARTE DE LA PANTALLA
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight * 1/4);") #3/4
        elif(i == 1): #Finalmente, ira a los 3/4 partes de la pantalla
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight * 1/2);")
        else:
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight * 2/3);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
            
        # Calculate new scroll height and compare with last scroll height

In [4]:
def iniciarSesion(browser):
    browser.get('https://www.linkedin.com/uas/login')
    file = open('config.txt')
    lines = file.readlines()
    username = lines[0]
    password = lines[1]


    elementID = browser.find_element_by_id('username')
    elementID.send_keys(username)

    elementID = browser.find_element_by_id('password')
    elementID.send_keys(password)
    elementID.submit()

In [5]:
# Exact Link Text
def busquedaPersonalizada(linkRef, name, browser, estado):
    linkRef = []
    if estado == 0:
        browser.get('https://www.linkedin.com/mwlite/search/results/all?keywords='+ name[0]+'+'+name[1]+'+'+name[2])
    else:
        browser.get('https://www.linkedin.com/mwlite/search/results/all?keywords='+ name[0]+'+'+name[1])
        
    #obtengo el link personal de cada perfil que se encuentre
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    
    if soup.find('ul', {'class': 'search-results'}) != None:
        name_div = soup.find('ul', {'class': 'search-results'})
        div = name_div.find_all('div',{'class': 'flex-entity-lockup'})

        p = name_div.find_all('p',{'class': 'subline-level-2 t-12 t-black--light t-normal search-result__truncate'})
        if p == []:
            p =  name_div.find_all('dd',{'class':'location small'})
        cont = 0
        
        ##datosInfo = soup.find_all('div', {'class':'flex-wrapper'})
        ##for i in datosInfo:
          ##  dataLink = i.find('a',{ 'class': 'profile-path primary-details'} , href = True)
            ##linkRef.append(dataLink['href'])
            
        for div1 in div:
            dataLink = div1.find_all('a',{ 'class': 'profile-path primary-details'} , href = True)
            for a in dataLink:
                try:
                    if p[cont].get_text().strip().find('erú') != -1:
                        linkRef.append(a['href'])
                except Exception:
                    pass
            cont += 1   
        
        
    return linkRef
        

In [6]:

def mainExtraccion(linkRef, browser, estado, name,cont):

#link de cada perfil
    path = 'https://www.linkedin.com'
    browser.get(path+linkRef)     
    slideBar(3, browser)
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    
    
#variables
    info = []
    nombre = loc = profile_title = company_name = joining_date = college_name = carreer = job_title = ""
    
#datosPersonales
    name_div = soup.find('div', {'class': 'flex-1 mr5'})
    name_loc = name_div.find_all('ul')
    nombre = name_loc[0].find('li').get_text().strip()
    try:        
        loc = name_loc[1].find('li').get_text().strip()
        profile_title = name_div.find('h2').get_text().strip()

        ##verificacion de que el usuario sea exactamente el buscado
        if estado == 0:
            if verificacion(nombre, name[1],name[2]) == -1:
                loc = "NN" ##no habilitado
    except Exception:
        pass

#extraccion del area de trabajo en base a EN o AT:
    if profile_title.find(' en ') != -1 or profile_title.find(' at ') != -1:
        job_title = profile_title[profile_title.find(" en ")+4:]
        profile_title = profile_title[:profile_title.find(" en ")]
        
#experiencia

    try:
        exp_section = soup.find('section', {'id': 'experience-section'}).find('ul')
        li_tags = exp_section.find('div')
        a_tags = li_tags.find('a')
        if job_title == "" or profile_title == "--":
            job_title = a_tags.find('h3').get_text().strip()
            profile_title = job_title
        company_name = a_tags.find_all('p')[1].get_text().strip()
        #joining_date = a_tags.find_all('h4')[0].find_all('span')[1].get_text().strip()
    except Exception:
        pass

#educacion

    try:
        edu_section = soup.find('section', {'id': 'education-section'}).find('ul')     
        college_name = edu_section.find('h3').get_text().strip()
        #carreer = edu_section.find('p', {'class': 'pv-entity__secondary-title pv-entity__fos t-14 t-black t-normal'}).find_all('span')[1].get_text().strip()
    except Exception:
        pass
    
#agregamos campos
    if company_name == "":
        company_name = job_title
    
    info.append(nombre)
    info.append(loc) 
    info.append(profile_title)
    info.append(company_name) 
    info.append(job_title) 
    info.append(college_name) 
    #info.append(carreer) 
    if nombre ==  "":
        cont += 1
    return info 

In [48]:
def baseData():
    cnn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DB)
    cursor = cnn.cursor()

    cursor.execute("select top 30000 b.numdoc ,b.nombre_largo, a.id_cuenta, b.nombre1, b.apPaterno, b.apMaterno from "+TABLA_BUSCAR+" a left join "+TABLA_BUSCAR2 +" b on a.ID_cuenta = b.idencuenta WHERE PERIODO='202001' and Rev = 0 ")
    records = cursor.fetchall()
    cursor.close()
    return records

In [8]:
def tablaInsertar(i, info, estado): #
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DB)
    cursor = conn.cursor()
    conn.cursor().execute("insert into AV_BASE_LINKEDIN values (?,?,?,?,?,?,?,?)",(i[0],info[0],info[1], info[3], info[2], info[5], estado,HOY ) )
    conn.commit()
    conn.close()

In [9]:
def tablaInsertar2(i, info, estado, ref): #
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DB)
    cursor = conn.cursor()
    conn.cursor().execute("insert into av_baselinkedin_razonsocial values (?,?,?,?,?,?,?,?)",(i[0],info[0],info[1], info[2], info[3], estado,HOY, ref ) )
    conn.commit()
    conn.close()

In [10]:
def actualizarEstado(i): #cambiar estado de 0 a 1
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DB)
    cursor = conn.cursor()
    conn.cursor().execute("update "+TABLA_BUSCAR+" SET REV = 1 WHERE ID_CUENTA = "+str(i[2]))
    conn.commit()
    conn.close()

In [11]:
def verificacion(texto, a1, a2):
    nameT = a1 + ' '+a2
    nameT = nameT.lower()
    var = {'ó':'o', 'á': 'a',"í":"i","ú":"u","é":"e","-":" "}
    texto = texto.lower()
    for j,k in var.items():
        texto = texto.replace(j,k)    
    return texto.find(nameT)

In [18]:
""" 
####################reclamo############################
if __name__ == '__main__':
   
    #records = baseData()
    #browser = webdriver.Chrome('chromedriver.exe')
    #iniciarSesion(browser)   
    for i in records:
        name = [i[3]]+[i[4]]+[i[5]] #nombre completo
        estado = 0 #estado 0: busqueda precisa, 1: busqueda no precisa
        linkRef = busquedaPersonalizada(linkRef, name, browser,estado) #obtener los link de perfil personal del usuario 
        print (i) #imprimir datos del usuario
        actualizarEstado(i) #cambio de estado del parque para no volver a verlo en sgtes iteraciones
        if len(linkRef) == 0:
            estado = 1
            linkRef = busquedaPersonalizada(linkRef, name, browser,estado)
            
        #EXTRAER INFORMACION DE CADA PERFIL    
        for j in linkRef:                        
            info = mainExtraccion(j,browser, estado,name) #DATOS EXTRAIDOS
            print(info, estado)
            
            if info[1].find('erú') != -1 or info[1].find('eru') != -1: #VERIFICO NACIONALIDAD PERUANA
                tablaInsertar(i, info,estado)   #INSERTO EN LA TABLA
                print("CHECK")
                break; #ROMPO PARA PASAR AL SIGUIENTE USUARIO      
                """

' \n####################reclamo############################\nif __name__ == \'__main__\':\n   \n    #records = baseData()\n    #browser = webdriver.Chrome(\'chromedriver.exe\')\n    #iniciarSesion(browser)   \n    for i in records:\n        name = [i[3]]+[i[4]]+[i[5]] #nombre completo\n        estado = 0 #estado 0: busqueda precisa, 1: busqueda no precisa\n        linkRef = busquedaPersonalizada(linkRef, name, browser,estado) #obtener los link de perfil personal del usuario \n        print (i) #imprimir datos del usuario\n        actualizarEstado(i) #cambio de estado del parque para no volver a verlo en sgtes iteraciones\n        if len(linkRef) == 0:\n            estado = 1\n            linkRef = busquedaPersonalizada(linkRef, name, browser,estado)\n            \n        #EXTRAER INFORMACION DE CADA PERFIL    \n        for j in linkRef:                        \n            info = mainExtraccion(j,browser, estado,name) #DATOS EXTRAIDOS\n            print(info, estado)\n            \n  

In [55]:
records2 = baseData()


In [56]:
#############################javaScript deshabilitado para que no realicen seguimiento de info##
if __name__ == '__main__': 
    records = records2
    HOY = datetime.now().strftime('%Y-%m-%d')
    #browser = webdriver.Chrome('chromedriver.exe')
    #iniciarSesion(browser)
    info = []
    contador = 0
    timeStop = 0
    SCROLL_PAUSE_TIME = 20
    
    for i in records:
        timeStop += 1
        if timeStop % 60 == 0:
            print("################ stop ####################")
            time.sleep(7)
            
        name = [i[3]]+[i[4]]+[i[5]] #nombre completo
        estado = 0 #estado 0: busqueda precisa, 1: busqueda no precisa
        linkRef = busquedaPersonalizada(linkRef, name, browser,estado) #obtener los link de perfil personal del usuario 
        print (i) #imprimir datos del usuario
        
        actualizarEstado(i)
        
        if len(linkRef) == 0:
            estado = 1
            linkRef = busquedaPersonalizada(linkRef, name, browser,estado)
            
        src = browser.page_source
        soup = BeautifulSoup(src, 'lxml')    
        datosInfo = soup.find_all('div', {'class':'flex-wrapper'})
        
        for j in datosInfo:
            nombreP = trabajo = lugarT = pais = ""
            nombreP = j.find('h3').get_text().strip()
            try:
                if nombreP[-1] == '1':
                    nombreP = nombreP[:nombreP.find('1')]
                if nombreP[-2] == '2':    
                    nombreP = nombreP[:nombreP.find('2')]
                if nombreP[-3] == '3':
                    nombreP = nombreP[:nombreP.find('3')]    
            except Exception:
                pass
            try:
                trabajo = j.find('div').get_text().strip()

                if trabajo.find(' en ') != -1:
                    lugarT = trabajo[trabajo.find(" en ")+4:]
                    trabajo = trabajo[:trabajo.find(" en ")]
                if trabajo.find(' at ') != -1:
                    lugarT = trabajo[trabajo.find(" at ")+4:]
                    trabajo = trabajo[:trabajo.find(" at ")]    
                if  trabajo.find('-') != -1  :
                    lugarT = trabajo[trabajo.find("-")+1:]
                    trabajo = trabajo[:trabajo.find("-")]  
            except Exception:
                pass
            
            ##link ref
            dataLink = j.find('a',{ 'class': 'profile-path primary-details'} , href = True)            
            
            try:
                pais = j.find('dd').get_text().strip()
                if pais.find('erú') != -1 or pais.find('eru') != -1: #VERIFICO NACIONALIDAD PERUANA
                    if estado == 0:
                        if verificacion(nombreP, name[1], name[2]) != -1:
                            
                            info = [nombreP,pais,lugarT,trabajo]
                            print(info ) 
                            #tablaE.loc[contador] = i[0],info[0],info[1], info[2], info[3], estado,HOY, ref 
                            contador += 1
                            print("CHECK", contador)
                            tablaInsertar2(i, info, estado, dataLink['href'])
                            
                            break
                    else:
                        
                        info = [nombreP,pais,lugarT,trabajo]
                        print(info)
                        tablaInsertar2(i, info, estado, dataLink['href'])
                        #tablaE.loc[contador] = i[0],info[0],info[1], info[2], info[3], estado,HOY, ref 
                        contador += 1
                        print("CHECK", contador)
                        break 
                            
            except Exception:
                pass
        
            
    print("fin")               

('06660175', 'ROMERO ROMERO LUIS ANTONIO', Decimal('327531'), 'LUIS', 'ROMERO', 'ROMERO')
('16712174', 'ROSARIO GIL SULMA AIDEE', Decimal('327536'), 'SULMA', 'ROSARIO', 'GIL')
('25647147', 'ANGELES VARGAS JULIO CESAR', Decimal('327542'), 'JULIO', 'ANGELES', 'VARGAS')
['Julio Angeles', 'Perú', '', 'Aduanas, Logística y Comercio exterior']
CHECK 1
('25564841', 'GARCIA ESQUIVEL FLABIO TORIBIO', Decimal('327554'), 'FLABIO', 'GARCIA', 'ESQUIVEL')
('80231517', 'SOTA PACHECO JANET ', Decimal('327570'), 'JANET', 'SOTA', 'PACHECO')
('45868555', 'CONZA ORE MARYELLA STEPHANY', Decimal('327593'), 'MARYELLA', 'CONZA', 'ORE')
('46984557', 'GUAYAC HUAMAN SEGUNDO VICTOR', Decimal('327614'), 'SEGUNDO', 'GUAYAC', 'HUAMAN')
('18161564', 'SANTOS GARCIA MARIA YOLANDA', Decimal('327617'), 'MARIA', 'SANTOS', 'GARCIA')
('40106396', 'LLATANCE LINDAO CRISTINA YOLANDA', Decimal('327621'), 'CRISTINA', 'LLATANCE', 'LINDAO')
('09304861', 'GIRALDO GIRALDO GLEDY GEORGINA', Decimal('327666'), 'GLEDY', 'GIRALDO', 'GIRA

('44642295', 'ALVAREZ FERRER LISSETH MILAGROS', Decimal('331779'), 'LISSETH', 'ALVAREZ', 'FERRER')
['Lisseth Alvarez', 'Perú', 'BACK OFFICE SOLUTION', 'Senior Help Desk']
CHECK 27
('09980114', 'MENDOZA CASTILLO ROBERTO ', Decimal('331782'), 'ROBERTO', 'MENDOZA', 'CASTILLO')
('10457526', 'CHONATE HUAMAN JOSE MANUEL', Decimal('331786'), 'JOSE', 'CHONATE', 'HUAMAN')
['JOSE ALBERTO MORALES CHONATE', 'Perú', 'ALA-CHICAMA', 'TECNICO DE CAMPO']
CHECK 28
('10300408', 'IPENZA GALVEZ DAVID RICARDO', Decimal('331805'), 'DAVID', 'IPENZA', 'GALVEZ')
['David Diaz Ipenza', 'Perú', '', 'Consultor MyContsys\nConsultor SAP B1 / Fi']
CHECK 29
('08491318', 'CCOYLLO CHINCHAY OLGA LOURDES', Decimal('326978'), 'OLGA', 'CCOYLLO', 'CHINCHAY')
('73088902', 'HERRERA GRIEVE PEDRO FERNANDO', Decimal('327020'), 'PEDRO', 'HERRERA', 'GRIEVE')
('42938340', 'TRUJILLO GARCIA CESAR ALFREDO', Decimal('327030'), 'CESAR', 'TRUJILLO', 'GARCIA')
('09140049', 'GONZALES NORIEGA ZADITH MARCELINA', Decimal('327035'), 'ZADITH', 'G

['Roxana Rodriguez Neyra', 'Perú', '', 'Coordinador de Bienestar Social']
CHECK 57
('40934962', 'VELIZ YANCE KENNETH ANDREI', Decimal('332359'), 'KENNETH', 'VELIZ', 'YANCE')
['Kenny Marvin Veliz', 'Perú', '-', '']
CHECK 58
('45072021', 'VERA VARGAS ANA BERTHA', Decimal('332362'), 'ANA', 'VERA', 'VARGAS')
['Ana Vera Vargas', 'Perú', 'Actividades  1.3 y 1.5  - Socios En Salud Sucursal Perú', 'Responsable']
CHECK 59
('46215360', 'GONZALES ROJAS JOSE WENCESLAO', Decimal('332364'), 'JOSE', 'GONZALES', 'ROJAS')
['Jose Antonio Gonzales Rojas', 'Perú', '', 'Especialista Managment Control External Business Units']
CHECK 60
('10474218', 'SANCHEZ HIDALGO OSCAR MANUEL', Decimal('332365'), 'OSCAR', 'SANCHEZ', 'HIDALGO')
['Oscar Sánchez', 'Perú', 'BISA Ingeniería de Proyectos S.A.', 'Ingeniero de Control de Proyectos Senior']
CHECK 61
('07451890', 'BALDEON NIETO JOSE LUIS', Decimal('332369'), 'JOSE', 'BALDEON', 'NIETO')
['José Baldeón', 'Perú', '', 'Director General NUEVAS SENDAS Centro de Psicotera

################ stop ####################
('09048816', 'ALLER LOJAS IDALIA MERCEDES', Decimal('329977'), 'IDALIA', 'ALLER', 'LOJAS')
('47703592', 'ABAD CRUZ MARIANO JORDY', Decimal('329981'), 'MARIANO', 'ABAD', 'CRUZ')
('33345157', 'PAREDEZ INFANTES ROSMERI BEATRIZ', Decimal('329993'), 'ROSMERI', 'PAREDEZ', 'INFANTES')
('46819624', 'PEÑA BALAREZO AZHELA ESTHER', Decimal('329995'), 'AZHELA', 'PEÑA', 'BALAREZO')
('10809915', 'COTERA POSTIGO MELISSA MARIBEL', Decimal('330007'), 'MELISSA', 'COTERA', 'POSTIGO')
('40561165', 'LIVISI CARBAJAL AGNIBAL SERGIO', Decimal('330010'), 'AGNIBAL', 'LIVISI', 'CARBAJAL')
('45839817', 'OTAROLA TORRE ROXANA LIDIA', Decimal('330825'), 'ROXANA', 'OTAROLA', 'TORRE')
['Roxana Lidia Otárola Torre', 'Perú', '', 'Contador Público Titulado']
CHECK 86
('44544470', 'CELIS CANO YADIRA MELISSA', Decimal('330828'), 'YADIRA', 'CELIS', 'CANO')
['Yadira Melissa Celis Cano', 'Perú', 'el Organismo Supervisor de la Inversión en Energía y Minería - Osinergmin', 'Supervisora

('10044549', 'CAYO HERRERA DE GONZALES SARA VICTORIA', Decimal('331083'), 'SARA', 'CAYO', 'HERRERA DE GONZALES')
['Sara Cayo', 'Perú', '', 'Modista Guinza confecciones']
CHECK 120
('47407928', 'ROJAS CALAGUA MAYTE TAIRY', Decimal('331085'), 'MAYTE', 'ROJAS', 'CALAGUA')
['mayte rojas', 'Perú', '-', '']
CHECK 121
('26663506', 'LAYZA CIEZA DE ROY MONICA ISABEL', Decimal('331092'), 'MONICA', 'LAYZA', 'CIEZA DE ROY')
('46290734', 'NIZAMA VENTOCILLA DAYANA FRANCISCA', Decimal('331101'), 'DAYANA', 'NIZAMA', 'VENTOCILLA')
['Betsy Lisset Nizama Perez', 'Perú', 'Idat', 'Estudiante']
CHECK 122
('70450495', 'LOPEZ RIVERA CHRISTIAN ANIBAL', Decimal('331107'), 'CHRISTIAN', 'LOPEZ', 'RIVERA')
['Christian Anibal López Rivera', 'Perú', 'Tecno Fast Pêru', 'Arquitecto Jr.']
CHECK 123
################ stop ####################
('17923682', 'MORA CATALAN PEDRO AVILIO', Decimal('331114'), 'PEDRO', 'MORA', 'CATALAN')
['Pedro Mora', 'Perú', '-', '']
CHECK 124
('42308672', 'PEÑA CARDOZA PATRICIA ', Decimal('33

...

['Edgar Chavez', 'Perú', 'PALCON PERU', 'Mechanical Engineer']
CHECK 194
('45547503', 'LOPEZ ACASIETE YENIFFER PAOLA', Decimal('331893'), 'YENIFFER', 'LOPEZ', 'ACASIETE')
['yeniffer lopez acasiete', 'Perú', 'IFB CERTUS', 'Estudiante']
CHECK 195
('45836935', 'ALVAREZ PANDURO CRISTIAN ENRIQUE', Decimal('331895'), 'CRISTIAN', 'ALVAREZ', 'PANDURO')
['Cristian Ágreda Alvarez', 'Perú', 'PIL PERU', 'Analista de Cotizaciones']
CHECK 196
('40107580', 'JORGE BERROCAL ANA ELVA', Decimal('331898'), 'ANA', 'JORGE', 'BERROCAL')
('46921478', 'BUSTAMANTE ROJAS CARLOS AUGUSTO', Decimal('331936'), 'CARLOS', 'BUSTAMANTE', 'ROJAS')
('06693661', 'VERGARAY SANCHEZ LUIS ALBERTO', Decimal('332004'), 'LUIS', 'VERGARAY', 'SANCHEZ')
('16708080', 'RUIZ URBINA CESAR AUGUSTO', Decimal('332052'), 'CESAR', 'RUIZ', 'URBINA')
('45293257', 'NEIRA RAMIREZ HELGA MILAGROS', Decimal('332067'), 'HELGA', 'NEIRA', 'RAMIREZ')
('10465995', 'MACHACUAY CAMPOS ZOZIMO BERNABE', Decimal('332073'), 'ZOZIMO', 'MACHACUAY', 'CAMPOS')
('0

('18197578', 'GUTIERREZ SALDAÑA JUAN EDUARDO', Decimal('328655'), 'JUAN', 'GUTIERREZ', 'SALDAÑA')
('10724135', 'FLORES LOPEZ LAUDA MASSIEL', Decimal('328676'), 'LAUDA', 'FLORES', 'LOPEZ')
('45044500', 'TELLO LENGUA KATYSA VERONICA DEL ROCIO', Decimal('328700'), 'KATYSA', 'TELLO', 'LENGUA')
('80180929', 'BOTTON FERNANDEZ HENRY JAIR', Decimal('328735'), 'HENRY', 'BOTTON', 'FERNANDEZ')
('21885360', 'TELLO CASTILLO DAVID CARLOS', Decimal('328772'), 'DAVID', 'TELLO', 'CASTILLO')
('21137886', 'SALAZAR ASTETE JUAN PAUL', Decimal('328803'), 'JUAN', 'SALAZAR', 'ASTETE')
('10279748', 'ALMENARA HUERTA CARLOS EMILIO', Decimal('328847'), 'CARLOS', 'ALMENARA', 'HUERTA')
('10718570', 'PALACIOS CALIXTO JESSICA PATRICIA', Decimal('328856'), 'JESSICA', 'PALACIOS', 'CALIXTO')
('41186563', 'ROSSI CELIS MARIA SOLEDAD', Decimal('328869'), 'MARIA', 'ROSSI', 'CELIS')
('46414845', 'TAFUR TAFUR JOSE ELVIS', Decimal('328876'), 'JOSE', 'TAFUR', 'TAFUR')
('45604433', 'DE LA PUENTE MUÑOZ LUCERO ', Decimal('328877')

('43636543', 'NAVEA ROSAS JONATAN RICARDO', Decimal('329554'), 'JONATAN', 'NAVEA', 'ROSAS')
['Jonatan Ricardo Navea Rosas', 'Perú', '', 'ABOGADO']
CHECK 230
('29674744', 'QUISPE PERALTA LUZ MARIA', Decimal('329559'), 'LUZ', 'QUISPE', 'PERALTA')
['LUZ QUISPE', 'Cajamarca, Perú', 'everis', 'Analista de operaciones']
CHECK 231
('41691727', 'MAITA VILAVILA TSAI TANIA', Decimal('329567'), 'TSAI', 'MAITA', 'VILAVILA')
('40386550', 'CHIRINOS VIDAL CLAUDIA ISABEL', Decimal('329575'), 'CLAUDIA', 'CHIRINOS', 'VIDAL')
['Claudia Chirinos', 'Cajamarca, Perú', '', 'Fue a Universidad Nacional de San Agustin de Arequipa']
CHECK 232
('07156394', 'CASTILLEJO VILCARINO ARTURO LUIS', Decimal('330542'), 'ARTURO', 'CASTILLEJO', 'VILCARINO')
('42739991', 'PORTUONDO PAREDES VALERIA MERCEDES', Decimal('330628'), 'VALERIA', 'PORTUONDO', 'PAREDES')
('32923258', 'LLORCA LOPEZ ANGELA MARIA', Decimal('330738'), 'ANGELA', 'LLORCA', 'LOPEZ')
('17887991', 'ZARE REYES ROSA MERCEDES', Decimal('331120'), 'ROSA', 'ZARE', 

('16805186', 'PISFIL AHUMADA CESAR GLENNIS', Decimal('331714'), 'CESAR', 'PISFIL', 'AHUMADA')
['Jose Pisfil Ahumada', 'Perú', '', 'en']
CHECK 236
('40185170', 'HERENCIA ZAPATA KATTY MILAGRITOS', Decimal('331721'), 'KATTY', 'HERENCIA', 'ZAPATA')
('40712922', 'CHOQUE SANCHEZ WALTER IVAN', Decimal('331736'), 'WALTER', 'CHOQUE', 'SANCHEZ')
['Walter Iván Choque Sánchez', 'Perú', '', 'Ingeniería Industrial']
CHECK 237
('07639961', 'LEON CALDERON CARLOS LEONCIO', Decimal('331739'), 'CARLOS', 'LEON', 'CALDERON')
('08320138', 'FERNANDEZ VALENCIA ILDEFONSO EDUARDO', Decimal('331752'), 'ILDEFONSO', 'FERNANDEZ', 'VALENCIA')
['Clever Gregorio Fernandez Ildefonso', 'Perú', 'vidrios aluminios fernandez', 'Dueño principal']
CHECK 238
('09171160', 'APOLINARIO LAGOS YOLANDA CONCEPCION', Decimal('331753'), 'YOLANDA', 'APOLINARIO', 'LAGOS')
['yolanda apolinario apolinario', 'Perú', 'I.E. N° 2051 de El Progreso', 'Docente']
CHECK 239
('42489122', 'SALOME ELESCANO JANET ZULEMA', Decimal('331755'), 'JANET', 

['Bilma Flores Reyes', 'Perú', '-', '']
CHECK 268
('10544403', 'IZQUIERDO BARRAZA CESAR ERNESTO', Decimal('331000'), 'CESAR', 'IZQUIERDO', 'BARRAZA')
('25717714', 'USHIÑAHUA RODRIGUEZ EMERY ', Decimal('331001'), 'EMERY', 'USHIÑAHUA', 'RODRIGUEZ')
('07239417', 'FARRO JARA SUSANA MARLENE', Decimal('331005'), 'SUSANA', 'FARRO', 'JARA')
('72790421', 'ALARCON DE LA CRUZ KATHERINE ESTHEFANY', Decimal('331007'), 'KATHERINE', 'ALARCON', 'DE LA CRUZ')
('73255877', 'MEJIA PALOMINO ALEJANDRA NOHEMI', Decimal('331013'), 'ALEJANDRA', 'MEJIA', 'PALOMINO')
('44898209', 'VASQUEZ SALAS EMERSON ANTONIO', Decimal('331016'), 'EMERSON', 'VASQUEZ', 'SALAS')
('10791524', 'MONTOYA VALENCIA WALTER ROBERT', Decimal('331024'), 'WALTER', 'MONTOYA', 'VALENCIA')
('08232337', 'SCHILDER MONTJOY INGRID JEANNETTE', Decimal('331030'), 'INGRID', 'SCHILDER', 'MONTJOY')
('44791218', 'MOSTACERO DIAZ EDWIN RICHARD', Decimal('331036'), 'EDWIN', 'MOSTACERO', 'DIAZ')
('07482385', 'COLINA ESTACIO ROSARIO DEL PILAR', Decimal('331